In [1]:
import json
from tqdm import tqdm
import os
import random
import numpy as np
from itertools import chain, combinations
import matplotlib.pyplot as plt
import matplotlib as mpl
from multiprocessing import Pool

from util import get_star_cache, plot_score_against_star, scrape_stars

In [2]:
checkTypeRisk = {
    "Maintained": "High",
    "Dependency-Update-Tool": "High",
    "Binary-Artifacts": "High",
    "Branch-Protection": "High",
    "Code-Review": "High",
    "Signed-Releases": "High",
    "Token-Permissions": "High",
    "Vulnerabilities": "High",
    "Fuzzing": "Medium",
    "Packaging": "Medium",
    "Pinned-Dependencies": "Medium",
    "SAST": "Medium",
    "Security-Policy": "Medium",
    "CI-Tests": "Low",
    "CII-Best-Practices": "Low",
    "Contributors": "Low"
}

checkTypeWeights = {
    "Critical": 10,
    "High": 7.5,
    "Medium": 5,
    "Low": 2.5
}

use_check_types = ["Maintained", "Dependency-Update-Tool", "Branch-Protection", "Binary-Artifacts", "Vulnerabilities", "Code-Review"]

In [3]:
files = [f for f in os.listdir("./data") if f.startswith("dataset")][:7]
repos = []


print("Loading scorecard dataset from files...")
for file_path in tqdm(files):
    with open("./data/" + file_path) as f:
        lines = f.readlines()
        for line in lines:
            try:
                repos.append(json.loads(line))
            except:
                pass
print("TOTAL # repos:", len(repos))
random.seed(63)
random.shuffle(repos)

Loading scorecard dataset from files...


100%|██████████| 7/7 [00:50<00:00,  7.15s/it]


TOTAL # repos: 667596


In [4]:


def get_score(repo, apply_checks = []):
    score, total = 0,0
    for check in repo["checks"]:
        checkType = check["name"]
        severity = checkTypeRisk[checkType]
        if checkType not in apply_checks or int(check["score"]) < 0:
            continue
        score += int(check["score"]) * checkTypeWeights[severity]
        total += checkTypeWeights[severity]
    return score / total if total > 0 else None
    

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def scale_plot_size(factor=1.5):
    default_dpi = mpl.rcParamsDefault['figure.dpi']
    mpl.rcParams['figure.dpi'] = default_dpi*factor

def set_plot_font_size(size):
    plt.rc('font', size=size) #controls default text size
    plt.rc('axes', titlesize=size) #fontsize of the title
    plt.rc('axes', labelsize=size) #fontsize of the x and y labels
    plt.rc('xtick', labelsize=size) #fontsize of the x tick labels
    plt.rc('ytick', labelsize=size) #fontsize of the y tick labels
    plt.rc('legend', fontsize=size) #fontsize of the legend

In [5]:


scale_plot_size()
set_plot_font_size(4)

def show_distribution_by_check_type(repos):
    fig, ax = plt.subplots(4, 4)
    plt.subplots_adjust(wspace=0.4, hspace=1)
    fig.tight_layout()
    r, c = 0, 0
    for check_type in checkTypeRisk.keys():
        scores = [get_score(repo, apply_checks=[check_type]) for repo in repos]
        valid_scores = [s for s in scores if s is not None]
        ax[r][c].hist(valid_scores)
        ax[r][c].set_title(check_type)
        c = c + 1 if c < 3 else 0
        r = r + 1 if c == 0 else r
    plt.show()
#show_distribution_by_check_type(repos)


In [6]:

#SAST is invalid because looks for CodeQL.  If CodeQL is present then it grades how it is used.  If not present then ignored.
# Contributors are invalid because contributors must come from different organizations.  If not enough organizations then ignored.
# CI tests are invlaid because 

repos = repos[:10000]
# modify repos in place to add stars property
repos = scrape_stars([repos[i] for i in range(len(repos))])
high_star_repos = [r for r in repos if r["stars"] > 5]
print("# high star repos:", high_star_repos)
show_distribution_by_check_type(high_star_repos)


"""
star_cache = get_star_cache()
scores = [get_score(repo, apply_checks=use_check_types) for repo in high_score_repos]
stars = [star_cache[repo["repo"]["name"]] for repo in high_score_repos]
plot_score_against_star(scores, stars)


print("Fetching stars...")
#print(json.dumps(repos[-1], indent=2))
high_score_repos = scrape_stars([repo for repo in repos if get_score(repo)])
star_cache = get_star_cache()
stars = [star_cache[repo["repo"]["name"]] for repo in high_score_repos]

print("# repos with score >= 6:", len(high_score_repos))
print("total # of repos: ", len(repos))

plot_score_against_star([repo["score"] for repo in high_score_repos], stars)

"""

fetched 2:   0%|          | 2/10000 [00:00<32:38,  5.10it/s]

b'{"id":58073677,"node_id":"MDEwOlJlcG9zaXRvcnk1ODA3MzY3Nw==","name":"verne","full_name":"LookLikeAPro/verne","private":false,"owner":{"login":"LookLikeAPro","id":8433523,"node_id":"MDQ6VXNlcjg0MzM1MjM=","avatar_url":"https://avatars.githubusercontent.com/u/8433523?v=4","gravatar_id":"","url":"https://api.github.com/users/LookLikeAPro","html_url":"https://github.com/LookLikeAPro","followers_url":"https://api.github.com/users/LookLikeAPro/followers","following_url":"https://api.github.com/users/LookLikeAPro/following{/other_user}","gists_url":"https://api.github.com/users/LookLikeAPro/gists{/gist_id}","starred_url":"https://api.github.com/users/LookLikeAPro/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/LookLikeAPro/subscriptions","organizations_url":"https://api.github.com/users/LookLikeAPro/orgs","repos_url":"https://api.github.com/users/LookLikeAPro/repos","events_url":"https://api.github.com/users/LookLikeAPro/events{/privacy}","received_events_url":"https

fetched 4:   0%|          | 4/10000 [00:00<30:05,  5.54it/s]

b'{"id":339683913,"node_id":"MDEwOlJlcG9zaXRvcnkzMzk2ODM5MTM=","name":"ruex","full_name":"angular-rust/ruex","private":false,"owner":{"login":"angular-rust","id":78678821,"node_id":"MDEyOk9yZ2FuaXphdGlvbjc4Njc4ODIx","avatar_url":"https://avatars.githubusercontent.com/u/78678821?v=4","gravatar_id":"","url":"https://api.github.com/users/angular-rust","html_url":"https://github.com/angular-rust","followers_url":"https://api.github.com/users/angular-rust/followers","following_url":"https://api.github.com/users/angular-rust/following{/other_user}","gists_url":"https://api.github.com/users/angular-rust/gists{/gist_id}","starred_url":"https://api.github.com/users/angular-rust/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/angular-rust/subscriptions","organizations_url":"https://api.github.com/users/angular-rust/orgs","repos_url":"https://api.github.com/users/angular-rust/repos","events_url":"https://api.github.com/users/angular-rust/events{/privacy}","received_event

fetched 5:   0%|          | 5/10000 [00:00<30:27,  5.47it/s]

...

b'{"id":67599602,"node_id":"MDEwOlJlcG9zaXRvcnk2NzU5OTYwMg==","name":"logeye","full_name":"59fe/logeye","private":false,"owner":{"login":"59fe","id":19700675,"node_id":"MDEyOk9yZ2FuaXphdGlvbjE5NzAwNjc1","avatar_url":"https://avatars.githubusercontent.com/u/19700675?v=4","gravatar_id":"","url":"https://api.github.com/users/59fe","html_url":"https://github.com/59fe","followers_url":"https://api.github.com/users/59fe/followers","following_url":"https://api.github.com/users/59fe/following{/other_user}","gists_url":"https://api.github.com/users/59fe/gists{/gist_id}","starred_url":"https://api.github.com/users/59fe/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/59fe/subscriptions","organizations_url":"https://api.github.com/users/59fe/orgs","repos_url":"https://api.github.com/users/59fe/repos","events_url":"https://api.github.com/users/59fe/events{/privacy}","received_events_url":"https://api.github.com/users/59fe/received_events","type":"Organization","site_admin"

fetched 293:   8%|▊         | 772/10000 [01:01<30:54,  4.98it/s]

b'{"id":189719785,"node_id":"MDEwOlJlcG9zaXRvcnkxODk3MTk3ODU=","name":"thecaraxapi","full_name":"aghahuseynov/thecaraxapi","private":false,"owner":{"login":"aghahuseynov","id":22603943,"node_id":"MDQ6VXNlcjIyNjAzOTQz","avatar_url":"https://avatars.githubusercontent.com/u/22603943?v=4","gravatar_id":"","url":"https://api.github.com/users/aghahuseynov","html_url":"https://github.com/aghahuseynov","followers_url":"https://api.github.com/users/aghahuseynov/followers","following_url":"https://api.github.com/users/aghahuseynov/following{/other_user}","gists_url":"https://api.github.com/users/aghahuseynov/gists{/gist_id}","starred_url":"https://api.github.com/users/aghahuseynov/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/aghahuseynov/subscriptions","organizations_url":"https://api.github.com/users/aghahuseynov/orgs","repos_url":"https://api.github.com/users/aghahuseynov/repos","events_url":"https://api.github.com/users/aghahuseynov/events{/privacy}","received_eve

fetched 295:   8%|▊         | 774/10000 [01:02<30:00,  5.12it/s]

b'{"id":82131607,"node_id":"MDEwOlJlcG9zaXRvcnk4MjEzMTYwNw==","name":"forklift.js","full_name":"w-p/forklift.js","private":false,"owner":{"login":"w-p","id":8106907,"node_id":"MDQ6VXNlcjgxMDY5MDc=","avatar_url":"https://avatars.githubusercontent.com/u/8106907?v=4","gravatar_id":"","url":"https://api.github.com/users/w-p","html_url":"https://github.com/w-p","followers_url":"https://api.github.com/users/w-p/followers","following_url":"https://api.github.com/users/w-p/following{/other_user}","gists_url":"https://api.github.com/users/w-p/gists{/gist_id}","starred_url":"https://api.github.com/users/w-p/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/w-p/subscriptions","organizations_url":"https://api.github.com/users/w-p/orgs","repos_url":"https://api.github.com/users/w-p/repos","events_url":"https://api.github.com/users/w-p/events{/privacy}","received_events_url":"https://api.github.com/users/w-p/received_events","type":"User","site_admin":false},"html_url":"https